In [1]:
import numpy as np
import tables
import os
import pytz
from IPython.display import clear_output
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import keras.backend as K
import datetime
import seaborn as sns

sns.set(font_scale=1.5)

Using TensorFlow backend.


In [6]:
class STFT(BaseModel):
    
    def __init__(self):
        
        super().__init__()

        # ========= Layer 1 =========
        self.lvl1_stride_x = 512
        self.lvl1_filter_height = 1
        self.lvl1_filter_width = 4096

        self.lvl1_filters_amount= 512 # or so called out-channels
        self.lvl1_regions_x = (WINDOW_SIZE - self.lvl1_filter_width)/self.lvl1_stride_x + 1 # 25
        self.lvl1_regions_y = self.lvl1_filters_amount # 512
        # the shape of wsin and wcos is (1, 4096, 1, 512) for now
        self.wsin = np.empty((self.lvl1_filter_height, self.lvl1_filter_width, 1, self.lvl1_filters_amount), dtype=np.float32)
        self.wcos = np.empty((self.lvl1_filter_height, self.lvl1_filter_width, 1, self.lvl1_filters_amount), dtype=np.float32)
        
        self.frequencies = np.logspace(np.log(self.start_freq),
                                       np.log(self.end_freq),
                                       self.lvl1_filters_amount,
                                       base=np.e,
                                       endpoint=True)
        
        self.discrete_time = np.arange(self.lvl1_filter_width)
        self.create_filters()
        self.wsin_var = tf.constant(value=self.wsin, dtype='float32', name="wsin")
        self.wcos_var = tf.constant(value=self.wcos, dtype='float32', name="wcos")
        print('wsin_var: {}'.format(self.wsin_var))
        print('wcos_var: {}'.format(self.wcos_var))

        
    def define_graph(self, activation=None):
        conv_sin = tf.nn.conv2d(input=self.input_X_ph,
                                filter=self.wsin_var,
                                strides=[1, 1, self.lvl1_stride_x, 1],
                                padding="VALID")
        conv_cos = tf.nn.conv2d(input=self.input_X_ph,
                                filter=self.wcos_var,
                                strides=[1, 1, self.lvl1_stride_x, 1],
                                padding="VALID")

        fourier_values_module_square = tf.square(conv_sin) + tf.square(conv_cos)

        # Output tensor the same shape as inputs except the last dimension is of size units.
        dense_layer = tf.layers.Dense(units=MIDI_PITCH_AMOUNT, activation=activation)
        # dense_layer = tf.layers.Dense(units=MIDI_PITCH_AMOUNT)

        batch_size_ph = tf.shape(self.input_X_ph)[0]
        
        self.pred_y_ph = dense_layer(tf.reshape(tf.log(fourier_values_module_square + 10e-12),
                                                shape=[batch_size_ph, tf.to_int32(self.lvl1_regions_x * self.lvl1_filters_amount)]))
        
        self.loss = tf.reduce_mean(tf.reduce_sum((self.input_y_ph - self.pred_y_ph) ** 2, axis=1))
        
        y_pred_converted =  tf.cast(tf.math.round(self.pred_y_ph), tf.int32)
        
        self.recall = tf.metrics.recall(self.input_y_ph, y_pred_converted)
        self.precision = tf.metrics.precision(self.input_y_ph, y_pred_converted)
        self.f1_score = tf.contrib.metrics.f1_score(self.input_y_ph, y_pred_converted)
        
        # self.loss = -tf.reduce_mean(self.input_y_ph * tf.log(self.pred_y_ph + 1e-8) +\
        #                           (1 - self.input_y_ph) * tf.log(1 - self.pred_y_ph + 1e-8))
        # self.loss = f1_loss(self.input_y_ph, self.pred_y_ph)
        
        self.optimizer_step = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        # self.optimizer_step = tf.train.MomentumOptimizer(self.learning_rate, self.mom).minimize(self.loss)        

NameError: ignored